### 정규 표현식 활용

### (1) HTML 문서에서 URL Link 추출하기

In [19]:
import re

s = '''<a href="http://www.daum.net">daum</a>  
<a href='http://www.naver.com'>naver</a>  
<a href=http://www.chosun.com>chosun</a>  
<a href=http://www.chosun.com class>chosun</a>  
<a href="http://job.daum.net/ " target="new">  
<a href="http://go.daum.net/bin/go.cgi?relative=1&url=/Mail-bin/login_f.cgi%3Ferror%3Dlogin" class="tls0">  
'''  

# (1)'<a href=' 뒤에 큰 따옴표(" ") 가 오는 경우
p = re.compile('''href="([^"]*?)"''', re.I)
match = p.search(s,0)
url = match.groups()   #  매칭된 전체 그룸 문자열을 튜플 형식으로 반환
# print(url)
print(list(filter(None,url)))   # ['http://www.daum.net']

# (2) '<a href='뒤에 작은 따옴표(' ')가 오는 경우
p = re.compile('''href='([^']*?)\'''', re.I)
match = p.search(s,0)
url = match.groups()   #  매칭된 전체 그룸 문자열을 튜플 형식으로 반환
# print(url)
print(list(filter(None,url)))   # ['http://www.naver.com']

# (3) '<a href='뒤에 큰/작은 따옴표가 안나오고 마지막 공백이 오는 경우
p = re.compile('''href=([^'"]\S+?)[\s>]''', re.I)
match = p.search(s,0)
url = match.groups()   #  매칭된 전체 그룸 문자열을 튜플 형식으로 반환
# print(url)
print(list(filter(None,url)))   # ['http://www.chosun.com']

['http://www.daum.net']
['http://www.naver.com']
['http://www.chosun.com']


In [21]:
# 세개의 정규표현식을 '|'를 사용하여 하나로 표현 한 경우
p = re.compile('''href="([^"]*?)"|href='([^']*?)\'|href=([^'"]\S+?)[\s>]''')
pos = 0
i = 0
while True:
    #print(i,pos)
    i += 1
    match = p.search(s,pos)
    if match:
        url = match.groups()
        #print(url)
        pos = match.end()
        print(list(filter(None,url))[0])  # 리스트의 첫번째  요소접근
    else:
        break

# filter함수는 특정 조건으로 걸러서 걸러진 요소들로 iterator객체를 만들어서 리턴해줍니다.
# map함수와 사용 방법은 동일하나, 함수의 결과가 참인지 거짓인지에 따라, 해당 요소를 포함할지를 결정합니다.      
 

http://www.daum.net
http://www.naver.com
http://www.chosun.com
http://www.chosun.com
http://job.daum.net/ 
http://go.daum.net/bin/go.cgi?relative=1&url=/Mail-bin/login_f.cgi%3Ferror%3Dlogin


In [11]:
# filter 함수 사용법 설명
print(list(filter(None,(None,10,None))))
print(list(filter(None,(1,None,10))))
print(list(filter(None,(None,None,None))))

[10]
[1, 10]
[]


In [12]:
L = ['apple','banana','','grape']
list(filter(None,L)) # 리스트에서 의미 없는 값을 삭제할 대 사용
 # filter() 함수의 첫번째 인수로 None 객체를 사용하면 입력값을 진리값을 판별하는데 그대로 사용할 수 있다

['apple', 'banana', 'grape']

In [13]:
# filter함수의 필요성
target = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = []

def is_even(n):
      return True if n % 2 == 0 else False

for value in target:
    print(is_even(value))
        
    if is_even(value):
        result.append(value)

print(result)  # 출력결과 : [2, 4, 6, 8, 10]

False
True
False
True
False
True
False
True
False
True
[2, 4, 6, 8, 10]


In [14]:
# filter 함수 사용
target = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

def is_even(n):
    return True if n % 2 == 0 else False

result = filter(is_even, target)

print(result,list(result)) # 출력결과 : [2, 4, 6, 8, 10]

<filter object at 0x000001730C3A8A08> [2, 4, 6, 8, 10]


### (2) 어휘 분석기 구현

In [35]:
# 파이썬 소스코드를 키워드별로 분석해준다

import re
import keyword

# print('|'.join(keyword.kwlist))

# 정규식 리스트
tokenREList = [
    ('KEYWORD','|'.join(keyword.kwlist)), # 파이썬 키워드
    ('NAME',r'[a-zA-Z_]\w*'),             # 첫문자는 영문자나 밑줄로 시작,두번째부터는 문자는 영문자나 숫자나 밑줄로 시작,키워드가 아닌 변수나 함수
    ('NUMBER', r'\d+|\d+\.\d*|\.\d+'),    # 정수나 실수(소수점 사용), 정수 | 정수.정수 |.정수
    ('LPAREN', r'\('),                    # 왼쪽 인자
    ('RPAREN', r'\)'),                    # 오른쪽 인자
    ('PLUS', r'\+'),                      # 더하기
    ('COLON', r':'),                      # 콜론
    ('WHITESPACE', r'\s+')                # 공백
]

rexp = ['(%s)'%exp for tokName, exp in tokenREList ]
# print(rexp)

p = re.compile('|'.join(rexp))

def lex(s):
    pos = 0
    match = p.match(s,pos)
    i = 0
    while match:
        #print(i)
        i += 1
        
        token = match.groups()
        #print(token)
        
        pos = match.end()
        #print(pos)
        for k in range(len(token)) : # 8회
            if token[k] :
                break
        yield (tokenREList[k][0], token[k])
        match = p.match(s,pos)

if __name__ == '__main__':
    s = '''
    for i in range(10):
        print (i + 5)''' 
    for tok in lex(s):
        print(tok)
    

('WHITESPACE', '\n    ')
('KEYWORD', 'for')
('WHITESPACE', ' ')
('NAME', 'i')
('WHITESPACE', ' ')
('KEYWORD', 'in')
('WHITESPACE', ' ')
('NAME', 'range')
('LPAREN', '(')
('NUMBER', '10')
('RPAREN', ')')
('COLON', ':')
('WHITESPACE', '\n        ')
('NAME', 'print')
('WHITESPACE', ' ')
('LPAREN', '(')
('NAME', 'i')
('WHITESPACE', ' ')
('PLUS', '+')
('WHITESPACE', ' ')
('NUMBER', '5')
('RPAREN', ')')


In [ ]:
['a','b','c'] ==> 'a|b|c'

### (3) 미니 챗봇 구현